**Note:** This notebook is written in Julia, so the cells can't be executed in Google Colab

# **Lab 1: Matrix Algorithms**
**Anders Ågren Thuné**

# **Abstract**

Short summary of the lab report. State the objectives, methods used, main results and conlusions.

This report presents how a number of fundamental matrix and vector operations were
implemented in the Julia programming language. In addition to the standard inner
product, matrix-vector product and matrix-matrix product, a CRS sparse matrix
structure was implemented along with a sparse matrix-vector product. The operations
were implemented according to mathematical descriptions and pseudo-code algorithms,
and verified using test data to ensure their correctness. All tests succeeded without
any significant surprises.

# **About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license.

In [14]:
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019
# Anders Ågren Thuné (athune@kth.se)
# Johan Hoffman (jhoffman@kth.se)

# Code written by Anders Ågren Thuné based on the template by Johan Hoffman.

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them.

In [33]:
# Load neccessary modules.

# No modules necessary for this lab.

# **Introduction**

Matrix algorithms are central to many methods in scientific computing. Even basic
concepts such as matrix multiplication are rich enough to develop a wide range of
algorithmic skills. Additionally, structural properties in different classes of
matrices can often be utilized to derive more efficient methods (Golub, Van Loan, 2013).

In this report, three basic matrix and vector operations are implemented, namely:
 - The inner product induced by the Euclidean $l_2$-norm in $\mathbb{R}^n$
 - Matrix-vector multiplication
 - Matrix-matrix multiplication

Additionally, a Compressed Row Storage (CRS) sparse matrix data structure
and a function for matrix-vector multiplication using it are implemented.

The implementations are based on the lecture notes *Introduction to Scientific
Computing* (Hoffman, 2018), which both provide concise explanations and pseudo-code
pseudo-code algorithms to many of the central concepts. The programming language used
is Julia, as it's fast, has native support for vector and matrix types and a syntax
that lends itself to mathematical expressions.

# **Methods**

Describe the methods you used to solve the problem. This may be a combination of text, mathematical formulas (Latex), algorithms (code), data and output.

Given a norm $\left\lVert \cdot\right\rVert$, the induced inner product is defined by
$\left\lVert x\right\rVert = (x,x)^{1/2}$ (Chapter 1.1).
The $l_2$-norm is defined by $\left\lVert x\right\rVert_2 = {\displaystyle \Big(\sum_{i=1}^{n}x_i^2\Big)^{1/2}}$,
giving the induced inner product $(x,y) = {\displaystyle \Big(\sum_{i=1}^{n}x_i\cdoty_i\Big)^{1/2}}$
Using this, we can define the following function in Julia:

In [ ]:
"""
Calculate the inner product (dot product) of vectors x and y.
"""
innerprod(x::Vector, y::Vector) = sum(x[i]*y[i] for i in 1:length(x))

The matrix-vector and matrix-matrix products

In [ ]:
"""
Calculate the matrix-vector product of matrix A and vector y.
"""
matvecprod(A::Matrix, x::Vector) = sum(A[:,i]*x[i] for i in 1:length(x))
"""
Calculate the matrix-matrix product of matrices A and B.
"""
matmatprod(A::Matrix, B::Matrix) = [innerprod(A[i,:],B[:,j]) for i in 1:size(A)[1], j in 1:size(B)[2]]

matmatprod

In [1]:
"""
A data structure to store sparse matrices in CRS format
"""
struct SpMat
    val::Vector
    col_idx::Array{Integer}
    row_ptr::Array{Integer}
end

"""
Calculate the matrix-vector product of sparse matrix A and vector x
"""
function spmatvecmul(A::SpMat, x::Vector)
    n = length(A.row_ptr)-1
    b = zeros(n)
    for i = 1:n
        for j = A.row_ptr[i]:A.row_ptr[i+1]-1
            b[i]+=A.val[j]*x[A.col_idx[j]]
        end
    end
    b
end

spmatvecmul

# **Results**

Present the results. If the result is an algorithm that you have described under the *Methods* section, you can present the data from verification and performance tests in this section. If the result is the output from a computational experiment this is where you present a selection of that data.

In [27]:
x = [1,1,2]
y = [3,1,-1]
innerprod(x,y)

2

In [28]:
A = [1 0 0;
     0 0 -1;
     0 1 0]
x = [1,1,2]
matvecprod(A,x)

3-element Array{Int64,1}:
  1
 -2
  1

In [29]:
A = [1 0 0;
     0 cos(pi/4) sin(pi/4);
     0 -sin(pi/4) cos(pi/4)]
B = [1 0 0;
     0 cos(7pi/4) sin(7pi/4);
     0 -sin(7pi/4) cos(7pi/4)]
matmatprod(A,B)

3×3 Array{Float64,2}:
 1.0  0.0           0.0        
 0.0  1.0          -2.77556e-16
 0.0  2.77556e-16   1.0        

In [2]:
A = SpMat([3,2,2,1,2,2,1,3,2,4,4,1,2,3],
          [1,2,2,3,6,2,3,3,4,5,2,5,5,6],
          [1,3,6,8,11,13,15])
println("A.val: ", A.val)
println("A.col_idx: ", A.col_idx)
println("A.row_ptr: ", A.row_ptr)

A.val: 

[3, 2, 2, 1, 2, 2, 1, 3, 2, 4, 4, 1, 2, 3]
A.col_idx: 

Integer[1, 2, 2, 3, 6, 2, 3, 3, 4, 5, 2, 5, 5, 6]
A.row_ptr: Integer[1, 3, 6, 8, 11, 13, 15]


In [3]:
x = [0,0,0,1,-1,0.5]
spmatvecmul(A,x)

6-element Array{Float64,1}:
  0.0
  1.0
  0.0
 -2.0
 -1.0
 -0.5

# **Discussion**

The results show that the algorithms work correctly as far as the test data indicates,
which was to be expected considering that they are directly derived from well
established mathematical definitions.

# **References**

- Hoffman, J. 2018. *Introduction to Scientific Computing*
- Golub, Gene H. and Van Loan, Charles F. 2013. *Matrix Computations*. 4th ed. Baltimore: John Hopkins University Press.